In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


#data

In [0]:
NUM_CLASSES = 10

In [3]:
(x_train, y_train), (x_test, y_test)= cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [5]:
input_layer = Input(shape=(32,32,3))

conv_layer_1 = Conv2D(
    filters = 10,
    kernel_size = (4,4),
    strides= 2,
    padding = 'same'
)(input_layer)

conv_layer_2 = Conv2D(
    filters=20,
    kernel_size = (3,3),
    strides = 2,
    padding='same'
)(conv_layer_1)

flatten_layer = Flatten()(conv_layer_2)

output_layer = Dense(units =10, activation='softmax')(flatten_layer)

model = Model(input_layer, output_layer)

In [6]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 10)        490       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 20)          1820      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 15,120
Trainable params: 15,120
Non-trainable params: 0
_________________________________________________________________


In [7]:
#합성곱, 배치정규화, 드롭아웃 적용하기

input_layer = Input((32,32,3))

x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input_layer)
x=BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(input_layer)
x=BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
x=BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(input_layer)
x=BatchNormalization()(x)
x = LeakyReLU()(x)

x= Flatten()(x)

x= Dense(128)(x)
x= BatchNormalization()(x)
x=LeakyReLU()(x)
x= Dropout(rate = 0.5)(x)

x= Dense(10)(x)
output_layer = Activation('softmax')(x)
model = Model(input_layer,output_layer)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
#train
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, batch_size = 32, epochs=10, shuffle=True, validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 20s 397us/step - loss: 1.7179 - acc: 0.4026 - val_loss: 1.3970 - val_acc: 0.5078
Epoch 2/10
50000/50000 [==============================] - 13s 252us/step - loss: 1.4017 - acc: 0.5009 - val_loss: 1.6351 - val_acc: 0.4364
Epoch 3/10
50000/50000 [==============================] - 13s 252us/step - loss: 1.3097 - acc: 0.5347 - val_loss: 1.4425 - val_acc: 0.4956
Epoch 4/10
50000/50000 [==============================] - 13s 250us/step - loss: 1.2370 - acc: 0.5610 - val_loss: 1.4883 - val_acc: 0.4944
Epoch 5/10
50000/50000 [==============================] - 13s 261us/step - loss: 1.1797 - acc: 0.5846 - val_loss: 1.3701 - val_acc: 0.5047
Epoch 6/10
50000/50000 [==============================] - 13s 251us/step - loss: 1.1272 - acc: 0.6019 - val_loss: 1.1990 - val_acc: 0.5706
Epoch 7/10
50000/500

#analysis

In [10]:
model.evaluate(x_test,y_test, batch_size=1000)

10000/10000 [==============================] - 0s 24us/step


[1.4213244438171386, 0.522500005364418]

In [0]:
CLASSES = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])

preds = model.predict(x_test)
preds_single = CLASSES[np.argmax(preds, axis = -1)]
actual_single = CLASSES[np.argmax(y_test, axis = -1)]

In [12]:
import matplotlib.pyplot as plt

n_to_show = 10
indices = np.random.chocice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize = (15,3))
fig.subplots_adjust(hspace=0.4, wspace = 0.4)

for i, idx in enumerate(indices):
  img = x_test[idx]
  ax = fig.add_subplot(1,n_to_show, i+1)
  ax.axis('off')
  ax.test(0.5,-0.35, 'pred = ' + str(preds_single[idx]), fontsize=10, ha='conter', transform=ax.transAxes)
  ax.text(0.5,-0.7, 'act = ' + str(actual_single[idx], fontsize=10, ha = 'center', transform=ax.transAxes))
  ax.imshow(img)

AttributeError: ignored